# メモ
- 変数追加
    

# 特徴量
## pitch
- 試合内投球数
- イニング
- イニング内打席数
- 打席内投球数
- 投手投球左右
- 投手役割
- 打者打席左右
- 打者打順
- プレイ前アウト数
- プレイ前ボール数
- プレイ前ストライク数
- プレイ前走者状況

### 追加
- ホームチームID
- アウェイチームID
- 捕手
- 球場ID
- 試合種別詳細
- 表裏
- 打者チームID
- 打者守備位置
- 打者試合内打席数
- プレイ前ホームチーム得点数
- プレイ前アウェイチーム得点数
- 投手登板順
- 投手イニング投球数
- 投手試合内対戦打者数

## player
### 打者&投手
- 育成選手F
- 位置
- 身長
- 体重
- 年俸
- 出身国

### 追加
- ドラフト順位

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

df_tr_pitch = pd.read_csv('../data/train_pitch.csv')
df_tr_player = pd.read_csv('../data/train_player.csv')
df_te_pitch = pd.read_csv('../data/test_pitch.csv')
df_te_player = pd.read_csv('../data/test_player.csv')
df_smp = pd.read_csv('../data/sample_submit_ball_type.csv', header=None)

In [2]:
import sys
sys.path.append("..")

import tqdm
import time
import preprocess
import importlib
importlib.reload(preprocess)
cnvrt = preprocess.Converter()
df_tr = cnvrt.convert_df(df_tr_pitch, df_tr_player)

In [10]:
import re
re.match(r"^num_|^ord_|^cat_", "_num_alskdj") is None

True

In [13]:
features = []
for x in df_tr.keys():
    if re.match(r"^num_|^ord_|^cat_", x) is not None:
        features.append(x)
features

['num_data',
 'num_game_throw',
 'ord_inning',
 'num_inning_bat',
 'num_pitch_in_bat',
 'ord_pitching',
 'num_pitcher_vs',
 'num_pitching_game',
 'num_pitching_inning',
 'ord_bat',
 'num_batter_batting',
 'num_point_home',
 'num_point_away',
 'num_out',
 'num_ball',
 'num_strike',
 'cat_pitcher_lr',
 'cat_pitcher_role',
 'cat_bat_lr',
 'cat_runner_state',
 'cat_home_id',
 'cat_away_id',
 'cat_dome_id',
 'cat_game_detail',
 'cat_fb',
 'cat_pitcher_team_id',
 'cat_batter_team_id',
 'cat_batter_def_posi',
 'cat_catcher_id',
 'num_height_batter',
 'num_weight_batter',
 'ord_draft_batter',
 'num_annual_salary_batter',
 'cat_training_player_batter',
 'cat_position_batter',
 'cat_country_batter',
 'num_height_pitcher',
 'num_weight_pitcher',
 'ord_draft_pitcher',
 'num_annual_salary_pitcher',
 'cat_training_player_pitcher',
 'cat_position_pitcher',
 'cat_country_pitcher']

In [14]:
tr_X = df_tr[features]
tr_Y = df_tr["ball_type"]

In [15]:
from sklearn.model_selection import KFold

cv_num = 5
kf = KFold(n_splits=cv_num, shuffle=True)

kf_index = []
for tr_i, te_i in kf.split(tr_X):
    kf_index.append([tr_i, te_i])

In [19]:
from sklearn.metrics import log_loss # モデル評価用(logloss) 
import lightgbm as lgb #LightGBM

# params
params = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 8,
    "metric": "multi_logloss",
    "verbose": 2
}

# train
loss_list = []
models = []
for i, kf in enumerate(kf_index):
    train_x = tr_X.iloc[kf[0]]
    train_y = tr_Y.iloc[kf[0]]
    test_x = tr_X.iloc[kf[1]]
    test_y = tr_Y.iloc[kf[1]]
    
    
    train_data = lgb.Dataset(train_x, label=train_y)
    eval_data = lgb.Dataset(test_x, label=test_y)
    
    print(f"train {i}")
    gbm = lgb.train(
        params,
        train_data, 
        valid_sets=eval_data,
        num_boost_round=1000,
        early_stopping_round=5,
        verbose_eval=50
    )
    
    models.append(gbm)

train 0
[50]	valid_0's multi_logloss: 1.26144
[100]	valid_0's multi_logloss: 1.208
[150]	valid_0's multi_logloss: 1.18863
[200]	valid_0's multi_logloss: 1.17639
[250]	valid_0's multi_logloss: 1.16875
[300]	valid_0's multi_logloss: 1.16297
[350]	valid_0's multi_logloss: 1.15808
[400]	valid_0's multi_logloss: 1.15403
[450]	valid_0's multi_logloss: 1.1512
[500]	valid_0's multi_logloss: 1.14838
[550]	valid_0's multi_logloss: 1.14636
[600]	valid_0's multi_logloss: 1.14496
[650]	valid_0's multi_logloss: 1.14381
[700]	valid_0's multi_logloss: 1.14301
[750]	valid_0's multi_logloss: 1.1426
[800]	valid_0's multi_logloss: 1.14272
[850]	valid_0's multi_logloss: 1.14274
[900]	valid_0's multi_logloss: 1.14286
[950]	valid_0's multi_logloss: 1.14286
[1000]	valid_0's multi_logloss: 1.14315
train 1
[50]	valid_0's multi_logloss: 1.26654
[100]	valid_0's multi_logloss: 1.21409
[150]	valid_0's multi_logloss: 1.19428
[200]	valid_0's multi_logloss: 1.18201
[250]	valid_0's multi_logloss: 1.17337
[300]	valid_0'

# 予測

In [20]:
df_te = cnvrt.convert_df(df_te_pitch, df_te_player, isTrain=False)
te_X = df_te[features]

In [21]:
result = []
for i, model in enumerate(models):
    start = time.time()
    print(f"start model{i}")
    result.append(model.predict(te_X))
    print(f"end time : {time.time()-start}")

start model0
end time : 188.2377369403839
start model1
end time : 189.3208179473877
start model2
end time : 183.81879091262817
start model3
end time : 176.676118850708
start model4
end time : 199.81790328025818


In [22]:
from scipy.special import softmax
result = np.array(result)
final_result = np.sum(result, axis=0)
final_result = softmax(final_result, axis=1)

In [23]:
df_submit = pd.concat([df_smp[0], pd.DataFrame(final_result)], axis=1).reset_index(drop=True)
df_submit.columns = range(df_submit.shape[1])
df_submit.to_csv("submits/v3.csv", index=False, header=False)